In [1]:
using BoundaryValueDiffEq, ModelingToolkit, Plots
using ModelingToolkit: t_nounits as t, D_nounits as D

In [2]:
@constants begin
    # mode numbers of interest
    m = 2
    n = 1

    # device parameters
    R0 = 1

    # poloidal wave numbers 
    ky = 0.25
    km = (m - 1) * ky
    k = (m) * ky
    kp = (m + 1) * ky

    # toroidal wave number
    kz = 1 / R0
    kn = n * kz

    # combined wave numbers
    Km2 = km^2 + kn^2
    K2 = k^2 + kn^2
    Kp2 = kp^2 + kn^2

    # Zeta-components
    ζz = 1
    ζy = 1

    # Lundquist number
    S = 10

    # Alfvénic Mach number
    M = 1 / 2 #S^(-1 / 2)
end

@parameters begin
    f(t::Real)::Real = tanh(t)
    g(t::Real)::Real = -2 * sech(t)^2
    γ
end

@variables begin
    ψp(t::Real)::Real
    φp(t::Real)::Real
    #γ(t::Real)::Real
    #x(t::Real)::Real
end;

In [4]:
eqns = [
    #f ~ tanh(t)
    #g(t) ~ -2 * sech(t)^2
    γ * (D(D(φp(t))) - Kp2 * φp(t)) ~ -ky * (f(t) * (m + 1) + 1) * (D(D(ψp(t))) - Kp2 * ψp(t) - g(t) * ψp(t))
    γ * ψp(t) - ky * (f(t) * (m + 1) + 1) * φp(t) - (1 / S) * (D(D(ψp(t))) - Kp2 * ψp(t)) ~ 0
    #D(γ(t)) ~ 0.0
    #D(x(t)) ~ abs2(ψp(t))
]

2-element Vector{Equation}:
 (Differential(t)(Differential(t)(φp(t))) - Kp2*φp(t))*γ ~ -(Differential(t)(Differential(t)(ψp(t))) - Kp2*ψp(t) - g(t)*ψp(t))*ky*(1 + (1 + m)*f(t))
 (-Differential(t)(Differential(t)(ψp(t))) + Kp2*ψp(t)) / S + ψp(t)*γ - ky*(1 + (1 + m)*f(t))*φp(t) ~ 0

In [7]:
global L = 15.0
constr = [ψp(L) ~ 0.0, φp(L) ~ 0.0];

In [8]:
@mtkcompile bvp = System(
    eqns, t;
    constraints=constr
)

Model bvp:
Equations (4):
  4 standard: see equations(bvp)
Unknowns (4): see unknowns(bvp)
  φp(t)
  ψp(t)
  ψpˍt(t)
  φpˍt(t)
Parameters (12): see parameters(bvp)
  γ
  kz [defaults to 1 / R0]
  kn [defaults to kz*n]
  ky [defaults to 0.25]
  S [defaults to 10]
  Kp2 [defaults to kn^2 + kp^2]
  ⋮

In [ ]:
function sideband!(du, u, p, t)
    ϕp, ψp, ψp1, ϕp1 = u
    du[1] = ϕp1
    du[2] = ψp1
    du[3] = S*((Kp2/S)*ψp + γ*ψp - ky*(1+(1+m)*f(t))*ϕp)
    du[4] = ϕp2
    du[5] = (1/γ)*(-Kp2*γ*ϕp - (Kp2*ψp + g(t)*ψp - S((Kp2/S)*ψp + γ*ψp - ky*((1 + 1+m)*f(t))*ϕp))*ky(1+(1+m)*f(t)))
    du[6] = abs2(ψ) # x(t) = |ψ|^2
    du[7] = 0
end

In [15]:
calculate_massmatrix(bvp)

7×7 LinearAlgebra.Diagonal{Float64, Vector{Float64}}:
 1.0   ⋅    ⋅    ⋅    ⋅    ⋅    ⋅ 
  ⋅   1.0   ⋅    ⋅    ⋅    ⋅    ⋅ 
  ⋅    ⋅   1.0   ⋅    ⋅    ⋅    ⋅ 
  ⋅    ⋅    ⋅   1.0   ⋅    ⋅    ⋅ 
  ⋅    ⋅    ⋅    ⋅   0.0   ⋅    ⋅ 
  ⋅    ⋅    ⋅    ⋅    ⋅   1.0   ⋅ 
  ⋅    ⋅    ⋅    ⋅    ⋅    ⋅   1.0

In [16]:
unknowns(bvp)

7-element Vector{SymbolicUtils.BasicSymbolic{Real}}:
 φp(t)
 ψp(t)
 ψpˍt(t)
 φpˍt(t)
 φpˍtt(t)
 x(t)
 γ(t)

In [19]:
# pass uncertain parameters/initial conditions to solver via `guesses` so it knows it can change them
prob = BVProblem(
    bvp,
    [ψp(t) => 0.0, φp(t) => 0.0, x(t) => 0.0, γ(t) => 0.123, D(ψp(t)) => 1e-5, D(φp(t)) => 1e-5, D(D(φp(t))) => 0],
    (-L, L),
    #guesses=[D(ψp(t)) => 1e-5, D(φp(t)) => 1e-5],
    tstops=[0.0]
)

┌ Warning: The BVProblem is overdetermined. The total number of conditions (# constraints + # fixed initial values given by op) exceeds the total number of states. The BVP solvers will default to doing a nonlinear least-squares optimization.
└ @ ModelingToolkit ~/.julia/packages/ModelingToolkit/b28X4/src/problems/bvproblem.jl:31


BVProblem with uType Vector{Float64} and tType Float64. In-place: true
Non-trivial mass matrix: true
timespan: (-15.0, 15.0)
u0: 7-element Vector{Float64}:
 0.0
 0.0
 1.0e-5
 1.0e-5
 0.0
 0.0
 0.123

In [20]:
sol = solve(prob, MIRK4(), dt=0.1)
plot(sol, idxs=(0, 1), label="ψ")
plot!(sol, idxs=(0, 3), label="φ")

: 

In [ ]:
sol.prob.p[1]